# 🌍 Impact Predictor – Milestone 3 (UI + Backend Integration)

This notebook demonstrates the working **Impact Predictor Web UI** developed for Milestone 3 of the project.  
It integrates a React frontend with an Express backend to predict earthquake impact levels (Low / Medium / High) based on:
- Magnitude  
- Depth  
- Soil Type  
- Population Density  

---


In [4]:
# Backend running on port 5050
# Sample API test using curl
!curl -X POST http://localhost:5050/api/predict \
  -H "Content-Type: application/json" \
  -d '{"magnitude":6.5,"depth":15,"soilType":"sand","populationDensity":1500}'


{"score":64.1,"risk":"Medium"}

### 🎨 React UI Launch Instructions

To view the user interface:

1. Open a new terminal  
2. Run the React app:
   ```bash
   cd client
   npm start


## 🌾 Rural Area

### Input Form
![Rural Form](images/rural_form.png)



## 🏙️ Urban Area

### Input Form
![Urban Form](images/urban_form.png)

### Medium Risk Result
![Urban Medium](images/urban_medium.png)

### High Risk Result
![Urban High](images/urban_high.png)


## 🗺️ Map Visualization
The map below shows predicted impact zones and risk intensity based on input parameters like magnitude, soil type, and population density.

![Map View](images/map_view.png)



## 🔍 Risk Level Computation

The backend computes an **Impact Score (0–100)** using:
\[
\text{Score} = \text{Magnitude Factor} × \text{Depth Factor} × \text{Soil Factor} × \text{Population Factor}
\]

### Classification
| Range | Risk Level | Description |
|:------:|:------------|:-------------|
| 0 – 34 | 🟢 Low | Minimal surface damage, localized tremors |
| 35 – 69 | 🟠 Medium | Noticeable shaking, moderate infrastructure effect |
| 70 – 100 | 🔴 High | Severe impact, high urban damage potential |

**Example:**
- `Magnitude = 5, Depth = 60 km` → **Low**
- `Magnitude = 6, Depth = 30 km` → **Medium**
- `Magnitude = 7.5, Depth = 10 km` → **High**

---


In [8]:
from IPython.display import Code
Code(filename="server/routes/predict.js", language="javascript")


const express = require("express");
const router = express.Router();

/**
 * Compute a simple mock "impact score" based on earthquake parameters.
 * You can replace this logic with a real ML model later.
 */
function computeImpactScore({ magnitude, depth, soilType, populationDensity }) {
  // Normalize magnitude to 0–100 scale
  const magScore = (magnitude / 10) * 100;

  // Deeper earthquakes have less surface impact
  const depthFactor = Math.exp(-depth / 70);

  // Soil amplification: sand > clay > rock
  const soilFactor =
    soilType === "sand" ? 1.2 : soilType === "clay" ? 1.1 : 1.0;

  // Population density factor (logarithmic)
  const popFactor = Math.log10(Math.max(1, populationDensity)) / 3;

  // Combine the effects
  const score = Math.min(
    100,
    Math.max(0, magScore * depthFactor * soilFactor * (0.7 + 0.3 * popFactor))
  );

  return Math.round(score * 10) / 10; // round to one decimal
}

/**
 * Map score to qualitative risk level
 */
function riskLevel(score) {
  if (score >= 70) return "High";
  if (score >= 35) return "Medium";
  return "Low";
}

/**
 * POST /api/predict
 * Expects JSON body: { magnitude, depth, latitude, longitude, soilType, populationDensity }
 * Returns: { score, risk }
 */
router.post("/", (req, res) => {
  console.log("✅ Received POST /api/predict");
  console.log("Body received:", req.body);

  try {
    // Extract data from body
    const data = req.body;

    // Validate required fields
    const required = [
      "magnitude",
      "depth",
      "soilType",
      "populationDensity",
    ];
    for (const field of required) {
      if (data[field] === undefined || data[field] === null) {
        return res.status(400).json({ error: `Missing field: ${field}` });
      }
    }

    // Compute score and risk
    const score = computeImpactScore(data);
    const risk = riskLevel(score);

    // Respond with JSON
    res.json({ score, risk });
  } catch (err) {
    console.error("❌ Error while computing impact score:", err);
    res.status(500).send("Server Error");
  }
});

module.exports = router;

In [9]:
from IPython.display import Code
Code(filename="client/src/components/ImpactForm.js", language="javascript")


import React, { useState } from "react";
import "./ImpactForm.css"; // optional if you have a style file

export default function ImpactForm({ onResult }) {
  const [form, setForm] = useState({
    magnitude: 5.0,
    depth: 10,
    latitude: 0,
    longitude: 0,
    soilType: "rock",
    populationDensity: 1000,
  });

  const [loading, setLoading] = useState(false);
  const [error, setError] = useState("");

  // 🧩 Preset buttons for quick demo
  const handlePreset = (preset) => {
    if (preset === "urban") {
      setForm({
        magnitude: 7.2,
        depth: 15,
        latitude: 19.07,
        longitude: 72.87,
        soilType: "sand",
        populationDensity: 8000,
      });
    } else if (preset === "rural") {
      setForm({
        magnitude: 5.0,
        depth: 80,
        latitude: 34.5,
        longitude: 70.5,
        soilType: "rock",
        populationDensity: 200,
      });
    }
  };

  // 🖊️ Handle input updates
  function update(e) {
    const { name, value } = e.target;
    setForm((prev) => ({
      ...prev,
      [name]: name === "soilType" ? value : parseFloat(value),
    }));
  }

  // 🔁 Reset to defaults
  const handleReset = () => {
    setForm({
      magnitude: 5.0,
      depth: 10,
      latitude: 0,
      longitude: 0,
      soilType: "rock",
      populationDensity: 1000,
    });
    setError("");
  };

  // 🚀 Submit and validate
  async function submit(e) {
    e.preventDefault();
    setError("");

    // 🧠 Client-side validation
    if (form.magnitude < 0 || form.magnitude > 10) {
      setError("⚠️ Magnitude must be between 0 and 10.");
      return;
    }
    if (form.depth < 0) {
      setError("⚠️ Depth cannot be negative.");
      return;
    }
    if (form.populationDensity < 0) {
      setError("⚠️ Population density cannot be negative.");
      return;
    }
    if (form.latitude < -90 || form.latitude > 90) {
      setError("⚠️ Latitude must be between -90 and 90.");
      return;
    }
    if (form.longitude < -180 || form.longitude > 180) {
      setError("⚠️ Longitude must be between -180 and 180.");
      return;
    }

    setLoading(true);

    try {
      const res = await fetch("http://localhost:5050/api/predict", {
        method: "POST",
        headers: { "Content-Type": "application/json" },
        body: JSON.stringify(form),
      });

      if (!res.ok) {
        throw new Error(`Server Error: ${res.status}`);
      }

      const data = await res.json();
      console.log("✅ Prediction Response:", data);

      // 🗺️ Attach coordinates for map
      data.latitude = form.latitude;
      data.longitude = form.longitude;

      onResult(data);
    } catch (err) {
      console.error("❌ Fetch Error:", err);
      setError("Failed to connect to server. Please ensure backend is running.");
    } finally {
      setLoading(false);
    }
  }

  return (
    <form className="impact-form" onSubmit={submit}>
      <h2>🌍 Earthquake Impact Predictor</h2>

      {/* Preset Buttons */}
      <div className="preset-buttons">
        <button type="button" onClick={() => handlePreset("urban")}>
          Urban Earthquake
        </button>
        <button type="button" onClick={() => handlePreset("rural")}>
          Rural Earthquake
        </button>
      </div>

      <label title="Magnitude measures the energy released by the earthquake.">
        Magnitude
        <input
          type="number"
          name="magnitude"
          value={form.magnitude}
          onChange={update}
          step="0.1"
          required
        />
      </label>

      <label title="Depth is how far underground the quake occurred — shallow quakes cause stronger shaking.">
        Depth (km)
        <input
          type="number"
          name="depth"
          value={form.depth}
          onChange={update}
          required
        />
      </label>

      <label title="Latitude indicates the north–south position of the earthquake’s epicenter.">
        Latitude
        <input
          typ

### 🚀 Direct Launch
- 🌐 [Open Impact Predictor UI](http://localhost:3000)
- 🖥️ Backend API Endpoint: [http://localhost:5050/api/predict](http://localhost:5050/api/predict)
